In [3]:
#@title Monte o Google Drive
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
#@title Instalar Modulos necessarios 

!pip install selenium
!apt install chromium-chromedriver
from google.colab import output; import os
output.clear()


In [ ]:
#@title Downloader Printables

URL_MODELS = 'https://www.printables.com/social/61-fotis-mint/models' #@param {type:"string"}
SAVE_PATH = '/content/drive/MyDrive/Modelos3D' #@param {type:"string"}
from google.colab import output; import os

os.chdir(SAVE_PATH)

import requests
import shutil
from time import sleep
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome import service as fs
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException

mobile_emulation = {
  "deviceMetrics": { "width": 375, "height": 812, "pixelRatio": 3.0 },
  "userAgent": "Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexus 5 Build/JOP40D) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.166 Mobile Safari/535.19",
  "touch": "true"
}

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)
browser = webdriver.Chrome(chrome_options=chrome_options)
browser.get(URL_MODELS)

WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, r"//button[contains(text(), 'I am OK with that')]"))).click()

while True:
  browser.execute_script("window.scrollBy(0,10000)") 
  try:
    WebDriverWait(browser, 2).until(EC.presence_of_element_located((By.XPATH, r"//span[contains(text(), 'All models loaded')]")))
    break
  except:
    pass

link_modelos = []
modelos = browser.find_elements(By.XPATH, "//a[@class='link clamp-two-lines']")
for modelo in modelos:
  print(modelo.get_attribute('href'))
  link_modelos.append(modelo.get_attribute('href'))

output.clear()

link_arquivos = []
for arquivo in link_modelos:
  try:
    browser.get(arquivo)

    try:
      WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, r"//span[contains(text(), 'Download')]"))).click()
    except:
      pass

    try:
      WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, r"//a[@class='btn btn-outline']")))
      arquivos = browser.find_elements(By.XPATH, "//a[@class='btn btn-outline']")
      for arquivo in arquivos:
        print(arquivo.get_attribute('href'))
        link_arquivos.append(arquivo.get_attribute('href'))
    except:
      pass

  except:
    pass

output.clear()

for url in link_arquivos: 
  res = requests.get(url, stream=True)
  filename = url.split('/')[-1]
  if res.status_code == 200:
      with open(filename, 'wb') as fh:
          res.raw.decode_content
          shutil.copyfileobj(res.raw, fh)

output.clear()

print("Download Finalizado, verifique na pasta informada")
%ls

In [5]:
#@title Teste de Velocidade 
!pip install speedtest-cli
from google.colab import output; import speedtest

s = speedtest.Speedtest()
s.get_servers()
s.get_best_server()

ping = round(s.results.ping)
velocidade_download = round(s.download(threads=None)*(10**-6))
velocidade_upload = round(s.upload(threads=None)*(10**-6))

output.clear()

print('PING : {} ms\nDOWNLOAD : {} Mbps\nUPLOAD : {} Mbps'.format(ping, velocidade_download, velocidade_upload))

PING : 18 ms
DOWNLOAD : 1674 Mbps
UPLOAD : 754 Mbps
